# Pipeline (YOLOv3 + CRAFT + CRNN) Metrics

In [1]:
import os
import sys
import cv2
import time
import pickle as pkl
import textdistance as td
import xml.etree.ElementTree as ET

In [2]:
import numpy as np
from itertools import zip_longest

In [4]:
from IPython.display import Image
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,15)

In [5]:
from src.Pipeline import Pipeline
import src.Metrics as metrics
import src.CraftTools as crafttools
import src.YOLOBbox as yolobbox
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [ ]:
Image(filename='doc/text-distance-table.png') 

## MERCOSUR (EU Format) Rainy

### 1. Dataset & annotations

In [6]:
image_dir = 'dev/mercosur/rainy/'
annot_dir='dev/mercosur_annotations/rainy/'

In [7]:
files_path = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
files_path.sort()
annot_path = [os.path.join(annot_dir, f) for f in os.listdir(annot_dir) if os.path.isfile(os.path.join(annot_dir, f))]
annot_path.sort()

In [8]:
len(files_path)

45

### 2. Pipeline

In [9]:
np.random.seed(14)
print('Creating pipeline...')
start_time = time.time()
pipeline = Pipeline()
end_time = time.time()
print('Pipeline created in: {} seconds\n'.format(end_time-start_time))

Creating pipeline...
Pipeline created in: 18.328786373138428 seconds



### 3. Predictions

In [10]:
lp_predictions = []
for file in files_path:
    print('Performing predictions on: {}'.format(file))
    start_time = time.time()
    lp_boxes, txt_pred = pipeline.image([file], syntax_analyzer=True)
    end_time = time.time()
    print('Total prediction time: {} seconds'.format(end_time-start_time))
    print('Printing outputs and text predictions...')
    # # Bounding Boxes
    for boxes, txt in zip(lp_boxes, txt_pred):
#         output = yolobbox.draw_boxes(cv2.imread(file), boxes, ['licence_plate'])
#         plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
#         plt.axis('off')
#         plt.show()
        print('Predictions: {}'.format(txt))
        lp_predictions.append(txt) 

Performing predictions on: dev/mercosur/rainy/rainy_1.jpg
Total prediction time: 5.706228256225586 seconds
Printing outputs and text predictions...
Predictions: [['aa', '388', 'jg']]
Performing predictions on: dev/mercosur/rainy/rainy_10.jpg
Total prediction time: 0.6102359294891357 seconds
Printing outputs and text predictions...
Predictions: [['']]
Performing predictions on: dev/mercosur/rainy/rainy_11.jpg
Total prediction time: 0.5539572238922119 seconds
Printing outputs and text predictions...
Predictions: [['']]
Performing predictions on: dev/mercosur/rainy/rainy_12.jpg
Total prediction time: 3.938347339630127 seconds
Printing outputs and text predictions...
Predictions: [['ad', '812', 'mv']]
Performing predictions on: dev/mercosur/rainy/rainy_13.jpg
Total prediction time: 1.2679047584533691 seconds
Printing outputs and text predictions...
Predictions: [['']]
Performing predictions on: dev/mercosur/rainy/rainy_14.jpg
Total prediction time: 1.577779769897461 seconds
Printing output

In [11]:
lp_predictions

[[['aa', '388', 'jg']],
 [['']],
 [['']],
 [['ad', '812', 'mv']],
 [['']],
 [['ac', '371', 'le']],
 [['']],
 [['']],
 [['aa', '624', 'vj']],
 [['aa', '624', 'vj']],
 [['aa', '205', 'nx']],
 [['aa', 'ooo', 'aa']],
 [['']],
 [['ab', '372', 'jn']],
 [['53zzey']],
 [['']],
 [['ab', '901', 'kg']],
 [['ad', '581', 'gr']],
 [['aazzssm']],
 [['']],
 [['ad', '550', 'og']],
 [['']],
 [['']],
 [['aa', '615', 'wb']],
 [['ab', '31', 'ku']],
 [['ab', '556z3']],
 [['sazsi']],
 [['cc', '010']],
 [['']],
 [['ad', '126', 'e']],
 [['ab', '775', 'nq']],
 [['lad', '288', 'so']],
 [['ae', '129', 'd']],
 [['ac', 'zns', 'yc']],
 [['']],
 [['']],
 [['da', '483', 'sz']],
 [['aa', '071', 'fu']],
 [['aa', 'lllhi']],
 [['cade', 'teleferico', 'otte']],
 [['ad', '81', 'ok']],
 [['aa', '778', 'aq']],
 [['']],
 [['205', 'no']],
 [['a', '215', 'o']]]

In [12]:
rainy = []
for image in lp_predictions:
    image_lps = []
    for lp in image:
        lp_text = ''.join(lp)
        image_lps.append(lp_text.upper())
    rainy.append(image_lps)

In [13]:
rainy

[['AA388JG'],
 [''],
 [''],
 ['AD812MV'],
 [''],
 ['AC371LE'],
 [''],
 [''],
 ['AA624VJ'],
 ['AA624VJ'],
 ['AA205NX'],
 ['AAOOOAA'],
 [''],
 ['AB372JN'],
 ['53ZZEY'],
 [''],
 ['AB901KG'],
 ['AD581GR'],
 ['AAZZSSM'],
 [''],
 ['AD550OG'],
 [''],
 [''],
 ['AA615WB'],
 ['AB31KU'],
 ['AB556Z3'],
 ['SAZSI'],
 ['CC010'],
 [''],
 ['AD126E'],
 ['AB775NQ'],
 ['LAD288SO'],
 ['AE129D'],
 ['ACZNSYC'],
 [''],
 [''],
 ['DA483SZ'],
 ['AA071FU'],
 ['AALLLHI'],
 ['CADETELEFERICOOTTE'],
 ['AD81OK'],
 ['AA778AQ'],
 [''],
 ['205NO'],
 ['A215O']]

In [15]:
with open('dev/metrics/rainy.data', 'wb') as file:  
    pkl.dump(rainy, file)

### 4. Get GT

In [14]:
gt = []
for file in annot_path:
    tree = ET.parse(file)
    root = tree.getroot()
    lp_text = []
    for lp in root.iter('object'):
        lp_text.append(lp.find('text').text)
    gt.append(lp_text)

In [15]:
gt

[['AA388JG'],
 ['AA205NO'],
 ['AD448OB'],
 ['AD812MV'],
 ['AD507QJ'],
 ['AC371LE'],
 ['AA924TF'],
 ['AD537EI'],
 ['AA624VJ'],
 ['AA624VJ'],
 ['AA205NX'],
 ['AA000AA'],
 ['AA015CR'],
 ['AB372JN'],
 ['AD537EY'],
 ['AA778AN'],
 ['AB801KG'],
 ['AD581GR'],
 ['AA228SM'],
 ['AA091JH'],
 ['AD550OG'],
 ['AC552XB'],
 ['AA018XI'],
 ['AA695WB'],
 ['AB371KU'],
 ['AB356ZY'],
 ['AA583SJ'],
 ['AC630IY'],
 ['AE197ZL'],
 ['AD126YE'],
 ['AB775NQ'],
 ['AD283SO'],
 ['AE129DD'],
 ['AC345XG'],
 ['AC256OZ'],
 ['AD277YC'],
 ['AA483SZ'],
 ['AA079FU'],
 ['AA411HJ'],
 ['AB661LS'],
 ['AD431QK'],
 ['AA778AQ', 'AA736KL'],
 ['AA736KL'],
 ['AA205NO'],
 ['AA205NO']]

In [18]:
with open('dev/metrics/gt.data', 'wb') as file:  
    pkl.dump(gt, file)

### 5. Metrics (Normal)

In [16]:
metrics.evaluate(ground_truth=gt, predictions=rainy)

[('AA388JG', 'AA388JG')]
[('AA205NO', '')]
[('AD448OB', '')]
[('AD812MV', 'AD812MV')]
[('AD507QJ', '')]
[('AC371LE', 'AC371LE')]
[('AA924TF', '')]
[('AD537EI', '')]
[('AA624VJ', 'AA624VJ')]
[('AA624VJ', 'AA624VJ')]
[('AA205NX', 'AA205NX')]
[('AA000AA', 'AAOOOAA')]
[('AA015CR', '')]
[('AB372JN', 'AB372JN')]
[('AD537EY', '53ZZEY')]
[('AA778AN', '')]
[('AB801KG', 'AB901KG')]
[('AD581GR', 'AD581GR')]
[('AA228SM', 'AAZZSSM')]
[('AA091JH', '')]
[('AD550OG', 'AD550OG')]
[('AC552XB', '')]
[('AA018XI', '')]
[('AA695WB', 'AA615WB')]
[('AB371KU', 'AB31KU')]
[('AB356ZY', 'AB556Z3')]
[('AA583SJ', 'SAZSI')]
[('AC630IY', 'CC010')]
[('AE197ZL', '')]
[('AD126YE', 'AD126E')]
[('AB775NQ', 'AB775NQ')]
[('AD283SO', 'LAD288SO')]
[('AE129DD', 'AE129D')]
[('AC345XG', 'ACZNSYC')]
[('AC256OZ', '')]
[('AD277YC', '')]
[('AA483SZ', 'DA483SZ')]
[('AA079FU', 'AA071FU')]
[('AA411HJ', 'AALLLHI')]
[('AB661LS', 'CADETELEFERICOOTTE')]
[('AD431QK', 'AD81OK')]
[('AA778AQ', 'AA778AQ'), ('AA736KL', 'AA778AQ')]
[('AA736KL', '